In [1]:
import d2lzh as d2l
from mxnet import nd
from mxnet.gluon import nn

In [2]:
# 填充 padding

# 定义一个函数计算卷积层。它初始化卷积层权重，并对输入和输出进行相应的升维和降维
def comp_conv2d(conv2d, X):
    conv2d.initialize()
    #(1,1)代表批量大小和通道数
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:]) #排除不关心的前两维：批量和通道

# 注意这里是两侧分别填充1行或列，所以在两侧一共填充2行或列
conv2d = nn.Conv2D(1, kernel_size=3, padding=1)
X = nd.random.uniform(shape=(8,8))
comp_conv2d(conv2d, X).shape

(8, 8)

In [3]:
# 步幅 stride
conv2d = nn.Conv2D(1, kernel_size=3, padding=1, strides=2)
print(comp_conv2d(conv2d, X).shape)


conv2d = nn.Conv2D(1, kernel_size=(3, 5), padding=(0, 1), strides=(3, 4))
print(comp_conv2d(conv2d, X).shape)

(4, 4)
(2, 2)


In [4]:
# 多输入通道

# 实现含多个输入通道的互相关运算
def corr2d_multi_in(X, K):
    # 首先沿着X和K的第0维（通道维）开始遍历
    # 然后使用*将结果列表变为add_n函数的位置参数（positional argument）来进行相加
    return nd.add_n(*[d2l.corr2d(x, k) for x, k in zip(X, K)])
# 另：列表前面加星号作用是将列表解开成两个独立的参数，传入函数
# data = [4,3],add(*data) equals to print add(4, 3)

X = nd.array([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
              [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = nd.array([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])

corr2d_multi_in(X, K)


[[  56.   72.]
 [ 104.  120.]]
<NDArray 2x2 @cpu(0)>

In [5]:
# 多输出通道

def corr2d_multi_in_out(X, K):
    # 对K的第0维遍历，每次同时输入X做互相关计算。所有结果使用stack函数合并在一起
    return nd.stack(*[corr2d_multi_in(X, k) for k in K])

#将核数组K同K+1（K中每个元素加一）和K+2连结在一起来构造一个输出通道数为3的卷积核
K = nd.stack(K, K + 1, K + 2)
print(K.shape)

corr2d_multi_in_out(X, K)

(3, 2, 2, 2)



[[[  56.   72.]
  [ 104.  120.]]

 [[  76.  100.]
  [ 148.  172.]]

 [[  96.  128.]
  [ 192.  224.]]]
<NDArray 3x2x2 @cpu(0)>

In [6]:
# 1×1的卷积层：
# 它的主要计算发生在通道维上，通常用来调整网络层之间的通道数，并控制模型复杂度。
# 假设我们将通道维当作特征维，将高和宽维度上的元素当成数据样本，
# 那么 1×1 卷积层的作用与全连接层等价。


#下面使用全连接层中的矩阵乘法来实现 1×1 卷积
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h*w))
    K = K.reshape((c_o, c_i))
    Y = nd.dot(K, X) #全连接层的矩阵乘法（注意K，X点乘顺序）
    return Y.reshape((c_o, h, w))

In [7]:
X = nd.random.uniform(shape=(3, 3, 3))
K = nd.random.uniform(shape=(2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

(Y1 - Y2).norm().asscalar() < 1e-6

# 经验证，做 1×1 卷积时，以上函数与之前实现的互相关运算函数corr2d_multi_in_out等价。

True